<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [ ]:
!pip install sqlalchemy==1.3.9


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [1]:
!pip install ipython-sql

In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas==1.1.5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
orange3 3.32.0 requires PyQt5!=5.15.1,>=5.12, which is not installed.
orange3 3.32.0 requires PyQtWebEngine>=5.12, which is not installed.
streamlit 1.26.0 requires pandas<3,>=1.3.0, but you have pandas 1.1.5 which is incompatible.
orange3 3.32.0 requires pandas>=1.3.0, but you have pandas 1.1.5 which is incompatible.
glue-core 1.0.1 requires jupyter-client<7, but you have jupyter-client 7.4.9 which is incompatible.


In [5]:
%sql sqlite:///my_data1.db

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [7]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [10]:
cur.execute("PRAGMA table_info(SPACEXTBL)")
column_names = [col[1] for col in cur.fetchall()]
print(column_names)

['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload', 'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome', 'Landing_Outcome']


In [11]:
unique_launch_sites = cur.execute("SELECT DISTINCT Launch_Site FROM SPACEXTBL").fetchall()

for site in unique_launch_sites:
    print(site[0])

CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [12]:
query = """
    SELECT *
    FROM SPACEXTBL
    WHERE Launch_Site LIKE 'CCA%'
    LIMIT 5
"""

# Execute the query
records_with_CCA = cur.execute(query).fetchall()

# Display the records
for record in records_with_CCA:
    print(record)

('2010-06-04', '18:45:00', 'F9 v1.0  B0003', 'CCAFS LC-40', 'Dragon Spacecraft Qualification Unit', 0, 'LEO', 'SpaceX', 'Success', 'Failure (parachute)')
('2010-12-08', '15:43:00', 'F9 v1.0  B0004', 'CCAFS LC-40', 'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese', 0, 'LEO (ISS)', 'NASA (COTS) NRO', 'Success', 'Failure (parachute)')
('2012-05-22', '7:44:00', 'F9 v1.0  B0005', 'CCAFS LC-40', 'Dragon demo flight C2', 525, 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt')
('2012-10-08', '0:35:00', 'F9 v1.0  B0006', 'CCAFS LC-40', 'SpaceX CRS-1', 500, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')
('2013-03-01', '15:10:00', 'F9 v1.0  B0007', 'CCAFS LC-40', 'SpaceX CRS-2', 677, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [13]:
query = """
    SELECT SUM(PAYLOAD_MASS__KG_)
    FROM SPACEXTBL
    WHERE Customer LIKE 'NASA (CRS)%'
"""

# Execute the query
total_payload_mass_NASA_CRS = cur.execute(query).fetchone()[0]
print(total_payload_mass_NASA_CRS)

48213


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [14]:
query = """
    SELECT AVG(PAYLOAD_MASS__KG_)
    FROM SPACEXTBL
    WHERE Booster_Version = 'F9 v1.1'
"""

# Execute the query
average_payload_mass_F9_v1_1 = cur.execute(query).fetchone()[0]
print(average_payload_mass_F9_v1_1)

2928.4


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [15]:
query = """
    SELECT Date
    FROM SPACEXTBL
    WHERE Landing_Outcome LIKE '%Success (ground pad)%'
    ORDER BY Date
    LIMIT 1
"""

# Execute the query
first_successful_landing_date = cur.execute(query).fetchone()[0]
print(first_successful_landing_date)

2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [16]:
query = """
    SELECT Booster_Version
    FROM SPACEXTBL
    WHERE Landing_Outcome LIKE '%Success (drone ship)%'
        AND PAYLOAD_MASS__KG_ > 4000
        AND PAYLOAD_MASS__KG_ < 6000
"""

# Execute the query
boosters_with_success_on_drone_ship = cur.execute(query).fetchall()

# Display the names of the boosters
for booster in boosters_with_success_on_drone_ship:
    print(booster[0])


F9 FT B1022
F9 FT B1026
F9 FT  B1021.2
F9 FT  B1031.2


### Task 7




##### List the total number of successful and failure mission outcomes


In [17]:
query = """
    SELECT Mission_Outcome, COUNT(*)
    FROM SPACEXTBL
    WHERE Mission_Outcome IN ('Success', 'Failure')
    GROUP BY Mission_Outcome
"""

# Execute the query
mission_outcomes_count = cur.execute(query).fetchall()

# Display the results
for outcome, count in mission_outcomes_count:
    print(f"{outcome}: {count}")

Success: 98


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [18]:
# Get the names of the booster_versions with the maximum payload mass
query = """
    SELECT Booster_Version
    FROM SPACEXTBL
    WHERE PAYLOAD_MASS__KG_ = (
        SELECT MAX(PAYLOAD_MASS__KG_)
        FROM SPACEXTBL
    )
"""

# Execute the query
max_payload_boosters = cur.execute(query).fetchall()

# Display the results
for booster in max_payload_boosters:
    print(booster[0])

F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2 
F9 B5 B1058.3 
F9 B5 B1051.6
F9 B5 B1060.3
F9 B5 B1049.7 


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [22]:
query = """
SELECT 
    substr(Date, 6, 2) AS Month,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM 
    SPACEXTBL
WHERE 
    substr(Date, 0, 5) = '2015' 
    AND Landing_Outcome = 'Failure (drone ship)'
"""
cur.execute(query)

# Fetch all the results
results = cur.fetchall()
print(results)

[('01', 'Failure (drone ship)', 'F9 v1.1 B1012', 'CCAFS LC-40'), ('04', 'Failure (drone ship)', 'F9 v1.1 B1015', 'CCAFS LC-40')]


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [23]:
query = """
SELECT
    Landing_Outcome,
    COUNT(*) AS Count
FROM
    SPACEXTBL
WHERE
    Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY
    Landing_Outcome
ORDER BY
    Count DESC;
"""
cur.execute(query)

# Fetch all the results
results = cur.fetchall()
print(results)

[('No attempt', 10), ('Success (drone ship)', 5), ('Failure (drone ship)', 5), ('Success (ground pad)', 3), ('Controlled (ocean)', 3), ('Uncontrolled (ocean)', 2), ('Failure (parachute)', 2), ('Precluded (drone ship)', 1)]


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
